In [6]:
import sys
print(sys.path)
sys.path.append('/Users/linhengyu/opt/anaconda3/lib/python3.8')

['/Users/linhengyu/Desktop', '/Users/linhengyu/opt/anaconda3/envs/python_tensor/lib/python38.zip', '/Users/linhengyu/opt/anaconda3/envs/python_tensor/lib/python3.8', '/Users/linhengyu/opt/anaconda3/envs/python_tensor/lib/python3.8/lib-dynload', '', '/Users/linhengyu/opt/anaconda3/envs/python_tensor/lib/python3.8/site-packages']


In [9]:
import pandas as pd
import re
#import cntext as ct
#import jieba
#import jieba.posseg as pseg
import tensorflow as tf

In [6]:
df1 = pd.read_csv('/Users/linhengyu/Documents/Python practice/ccClub_project_1600-1838.csv')
df2 = pd.read_csv('/Users/linhengyu/Documents/Python practice/ccClub_project_1400-1599.csv')
data = pd.concat([df1, df2], axis = 0)
data = data.reset_index(drop = True)

In [7]:
#將comment分割成list[[推噓],[帳號:],[留言],[日期],[時間],[空格],[IP]]
def split_text(row):
    pattern = re.compile(r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}')
    try:
        for i in range(2, len(row)):
            matches = pattern.search(row[i]).group()
            row[i] = row[i].replace(matches, "")
            row[i] = row[i].split(' ')
            row[i].append(matches)
            row[i][2:-3] = [' '.join(row[i][2:-3])]
    except:
        pass

In [8]:
data.apply(split_text, axis = 1)

0       None
1       None
2       None
3       None
4       None
        ... 
8734    None
8735    None
8736    None
8737    None
8738    None
Length: 8739, dtype: object

In [155]:
data.to_csv('/Users/linhengyu/Documents/Python practice/check_2.csv')

In [9]:
def clean_text(row):#將同一個帳號所講的話合併
    id = []
    for i in range(2, len(row)):
        try:
            id.append(row[i][1])
        except:
            pass
    u_id = list(set(id)) #建立帳號列表
    c = []
    for u in u_id:
        l = ''
        for i in range(2, len(row)):
            try:
                if u == row[i][1]:
                    l += row[i][2] #儲存組合後的comment
            except:
                pass
        c.append([u,l]) #輸出[帳號, 留言]
    return c

In [10]:
comment_dic = pd.DataFrame({})
comment_dic["comment"] = data.apply(clean_text, axis = 1)
comment_dic = pd.DataFrame([pd.Series(x) for x in comment_dic.comment])
comment_dic.columns = ['comment_{}'.format(x+1) for x in comment_dic.columns]
title_and_content = data[["title","content"]]
data_clean = pd.concat([title_and_content, comment_dic], axis = 1) 

In [11]:
#篩選真正在問問題的貼文
def ask_for_recommend(row):
    recommend_words = ["挑選","選擇","請教","推薦"]
    for r in recommend_words:
        if r in row["title"]:
            return r
        
#建立手術種類的column
def surgery_type(row):
    surgery_words = ["鼻",
                     "乳","曼陀","水滴","自體",
                     "埋線","玻尿酸","微整",
                     "抽脂","威塑","減脂",
                     "眼皮","淚溝","眼袋","眼袋",
                     "近視"
                     "削骨"
                     ]
    for s in surgery_words:
        if s in row["title"]:
            return s

#將手術種類分類
def surgery_location(row):
    if row ["surgery_type"] in ["鼻"]: return "nose"
    if row ["surgery_type"] in ["乳","曼陀","水滴","自體"]: return "breast"
    if row ["surgery_type"] in ["埋線","玻尿酸","微整"]: return "micro"
    if row ["surgery_type"] in ["抽脂","威塑","減脂"]: return "fat"
    if row ["surgery_type"] in ["眼皮","淚溝","眼袋","眼袋"]: return "eye"
    if row ["surgery_type"] in ["近視"]: return "shortsight"
    if row ["surgery_type"] in ["削骨"]: return "bone"

In [12]:
data_clean["recommend_type"] = data_clean.apply(ask_for_recommend, axis = 1)
data_clean = data_clean.dropna(subset=["recommend_type"])
data_clean["surgery_type"] = data_clean.apply(surgery_type, axis = 1)
data_clean = data_clean.dropna(subset=["surgery_type"])
data_clean["surgery_location"] = data_clean.apply(surgery_location, axis = 1)

In [13]:
#只留下前100個人的留言
data_clean = data_clean.drop(data_clean.columns[102:611], axis = 1)

In [228]:
data_clean.to_csv('/Users/linhengyu/Documents/Python practice/ccClub_project_ptt_data_1400-1838_v4.csv')

In [303]:
#人名辨識function
def isname(single_word_string):
    pair_word_list = pseg.lcut(single_word_string)
    for eve_word, cixing in pair_word_list:
        if cixing == "nr":
            return True
    return False

In [342]:
#句子切割+辨識醫師名稱
def doctor_name(row):
    doctor = []
    try:
        for i in range(2, len(row)):
            words = list(jieba.cut(row[i][1], cut_all = False))
            d = []
            for word in words:
                if isname(word) and len(word) == 3:
                    d.append(word)
                    row[i].append(word)
            if d != []:
                doctor.append(d)
    except:
        pass
    return doctor

In [16]:
data_clean.iloc[0][2]

['smilifly:', '三位醫師都諮詢過~但個人比較喜歡吳孟穎   的案例 線條漂亮外也沒有凹凸不平的問題    比價後也覺得是能接受的範圍   ']

In [23]:
#情緒辨識
text = '三位醫師都諮詢過~但個人比較喜歡吳孟穎   的案例 線條漂亮外也沒有凹凸不平的問題    比價後也覺得是能接受的範圍   '
ct.sentiment(text=text,
             diction=ct.load_pkl_dict('ChineseFinancialFormalUnformalSentiment.pkl'),
             lang='chinese')

{'ChineseFinancialFormalUnformalSentiment_num': 0,
 'Desc_num': 3,
 'Referer_num': 0,
 'stopword_num': 11,
 'word_num': 43,
 'sentence_num': 1}

In [280]:
#根據surgery_location分開每個data
surgery_location = ['nose','breast','fat','eye','shortsight','micro']
for s in surgery_location:
    globals()[f'{s}_data'] = data_clean[data_clean['surgery_location'].isin([s])]